---

### Adım Adım ResNeXt Grouped Conv Residual 

----

# Adım 1 — En sade iskelet: sadece “residual fikri”

* Bu sınıf şimdilik hiç öğrenilebilir katman içermiyor

In [1]:
import torch 
import torch.nn as nn

class ResnetxBlock(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        identity = x
        out = x
        out = out + identity
        return out

* Bu kod “residual”ın matematik iskeleti.
* Şimdi bu “out = x” kısmını gerçek bir F(x) yapacağız.

----
----

# 2) Adım 2 — F(x)’i “grouped 3×3” ile başlatalım (en sade öğrenilebilir F)

En minimal öğrenilebilir grouped residual:

>F(x) = 3×3 grouped conv + BN + ReLU (opsiyon)
>Skip = x (boyutlar aynıysa)

In [ ]:
import torch.functional as F
import torch 
import torch.nn as nn

class GroupedResidualBlock(nn.Module):
    def __init__(self, channels, groups=4):
        super().__init__()
        self.conv = nn.Conv2d(
            channels, channels, kernel_size=3, stride=1, padding=1,
            groups=groups, bias=False
        )
        self.bn = nn.BatchNorm2d(channels)

    def forward(self, x):
        skip = x
        out = self.conv(x)
        out = self.bn(out)
        out = F.relu(out, inplace=True)
        return out + skip

✅ Bu artık “grouped conv residual block”tır.

Ama ResNeXt tarzında genelde iki şey daha istenir:

* Kanalı ayarlayabilmek (in_ch ≠ out_ch)

* Downsample yapabilmek (stride=2)

* Bunu eklemek için projection skip gerekir.

-----
-----

# 3) Adım 3 — Boyutlar değişince skip’i düzenlemek (projection)

In [ ]:
import torch.functional as F
import torch 
import torch.nn as nn

class GroupedResidualBlock(nn.Module):
    def __init__(self,in_ch,out_ch,stride=1,groups=4):
        super().__init__()

        self.conv = nn.Conv2d(in_ch,out_ch,kernel_size=3,stride=stride,padding=1,groups=groups,bias = False)
        self.bn = nn.BatchNorm2d(out_ch)

        self.proj = None
        if stride != 1 or in_ch!=out_ch:
            self.proj = nn.Sequential(
                nn.Conv2d(in_ch,out_ch,kernel_size=1,stride=1,bias=False),
                nn.BatchNorm2d(out_ch))
            
    def forward(self,x):
        skip = x if self.proj is None else self.proj(x)

        out = self.conv(x)
        out = self.bn(out)
        out = F.relu(out,inplace=True)

        return out + skip

* ✅ Bu, “grouped conv residual block”u tam anlamıyla kullanılabilir hale getirir.

----
----

# 4) ResNeXt ile fark nerede?

ResNeXt’te “F(x)” sadece tek grouped 3×3 değildir; genelde:

* grouped 3×3’ün önünde/arkasında kanal düzenleyen 1×1 katmanlar vardır (bu sayede groups daha verimli kullanılır ve kapasite artışı daha kontrollü olur).

* Yani ResNeXt’in standart bloğu bu “sade versiyonun” genişletilmiş halidir.

### Şimdi kod (tam standart ResNeXt bloğu)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def conv1x1(in_ch, out_ch, stride=1):
    return nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride, bias=False)

def conv3x3(in_ch, out_ch, stride=1, groups=1):
    return nn.Conv2d(
        in_ch, out_ch, kernel_size=3, stride=stride, padding=1,
        groups=groups, bias=False
    )

class ResNeXtBlock_X(nn.Module):
    expansion = 4
    def __init__(self, in_ch, bottleneck_ch, stride=1, groups=32, base_width=4):
        super().__init__()

        # 1) Bloğun çıkış kanal sayısı (standart ResNeXt/ResNet genişletme kuralı)
        out_ch = bottleneck_ch * self.expansion

        # 2) Grouped 3×3'ün çalışacağı iç kanal (width)
        #    width = (per_group_width) * groups
        width = int(bottleneck_ch * (base_width / 64.0)) * groups

        # -------------------------
        # Main path: F(x)
        # -------------------------
        # (a) 1×1 reduce: in_ch -> width
        self.conv1 = conv1x1(in_ch, width)
        self.bn1 = nn.BatchNorm2d(width)

        # (b) 3×3 grouped: width -> width (stride burada uygulanır)
        self.conv2 = conv3x3(width, width, stride=stride, groups=groups)
        self.bn2 = nn.BatchNorm2d(width)

        # (c) 1×1 expand: width -> out_ch
        self.conv3 = conv1x1(width, out_ch)
        self.bn3 = nn.BatchNorm2d(out_ch)

        # -------------------------
        # Skip path: identity veya projection
        # -------------------------
        self.downsample = None
        if stride != 1 or in_ch != out_ch:
            self.downsample = nn.Sequential(
                conv1x1(in_ch, out_ch, stride=stride),
                nn.BatchNorm2d(out_ch)
            )

    def forward(self, x):
        # skip
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        # F(x)
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out, inplace=True)

        out = self.conv2(out)
        out = self.bn2(out)
        out = F.relu(out, inplace=True)

        out = self.conv3(out)
        out = self.bn3(out)

        out = out + identity
        out = F.relu(out, inplace=True)
        return out

In [ ]:
x = torch.randn(2, 256, 56, 56)

blk = ResNeXtBlock_X(in_ch=256, bottleneck_ch=64, stride=1, groups=4, base_width=4)
y = blk(x)

print("x:", x.shape)
print("y:", y.shape)

x: torch.Size([2, 256, 56, 56])
y: torch.Size([2, 256, 56, 56])


# Kullanım tablosu: “Ne verirsem ne olur?”

| Verdiğin parametre         | Ne işe yarar                            | Ne değişir?                                                                                 | Tipik değerler                         | Yan etki / dikkat                                                             |
| -------------------------- | --------------------------------------- | ------------------------------------------------------------------------------------------- | -------------------------------------- | ----------------------------------------------------------------------------- |
| `in_ch`                    | Bloğa giren feature map’in kanal sayısı | Skip ve ilk 1×1’in giriş boyutu                                                             | Stage’e göre: 64, 128, 256, 512, 1024… | `in_ch` yanlışsa blok bağlanmaz (shape mismatch)                              |
| `bottleneck_ch`            | Bloğun “temel kanal ölçeği” (planes)    | Çıkış kanalını belirler: `out_ch = bottleneck_ch*4` ve `width` hesabına girer               | 64, 128, 256, 512 (stage’e göre)       | Yanlış seçilirse block çıkışı beklenen stage kanalına uymaz                   |
| `stride`                   | Uzamsal downsample yapıp yapmama        | `H×W` düşer (stride=2) veya aynı kalır (stride=1)                                           | 1 veya 2                               | `stride=2` ise skip’e projection genellikle şart                              |
| `groups`                   | Cardinality (kaç grup)                  | Grouped 3×3’ün grup sayısı; grup sayısı artınca “paralel temsil” artar                      | 4, 8, 16, 32                           | `width` mutlaka `groups`’a bölünebilir olmalı (pratikte)                      |
| `base_width`               | Grup başına taban genişlik ölçeği       | `width = int(bottleneck_ch*(base_width/64))*groups` üzerinden iç genişliği büyütür/küçültür | 4 (klasik), bazen 8                    | Çok büyütürsen maliyet artar, çok küçültürsen kapasite düşer                  |
| `expansion` (sınıf sabiti) | Son 1×1’in genişletme katsayısı         | Çıkış kanal çarpanı                                                                         | Genelde 4                              | Çoğu ResNeXt/ResNet standardı 4’tür; değiştirirsen stage kanal düzeni bozulur |


## Örnek A — Boyutlar aynı kalsın (projection yok)

Amaç: Sadece blok eklemek, H×W değişmesin.

* stride = 1

* in_ch ile out_ch eşleşmeli

**out_ch=bottleneck_ch×4**

*O halde:*

**in_ch=bottleneck_ch×4**

#### Kullanım:
```python 
# in_ch=256 ise bottleneck_ch=64 seçersek out_ch=256 olur → projection gerekmez
ResNeXtBlock(in_ch=256, bottleneck_ch=64, stride=1, groups=4, base_width=4)
```


## Örnek B — Downsample  (projection var)

Amaç: H×W yarıya insin.

* stride = 2 (grouped 3×3 üzerinde uygulanır)

* Skip hattı: 1×1 conv (stride=2) ile küçültülür

#### Kullanım:
```python 
# HxW düşer, kanal out_ch olur
ResNeXtBlock(in_ch=256, bottleneck_ch=128, stride=2, groups=4, base_width=4)
```


#### Ne olur?

* out_ch = 128*4 = 512

* Çıkış shape: [N, 512, H/2, W/2]

* Skip: projection (1×1 + BN, stride=2)

## Örnek C — groups arttırınca ne olur?
* Amaç: Daha fazla “paralel temsil” (cardinality ↑)

```python 
ResNeXtBlock(in_ch=256, bottleneck_ch=64, stride=1, groups=8, base_width=4)
```

#### Ne olur?

* width hesabında *groups var → groups artınca width genelde artar

* Grouped 3×3 artık 8 gruba bölünür

* Kapasite artabilir; maliyet de artabilir (width’a bağlı)

-------
---------
------------
-----------

## Terimler Sözlüğü (Kısa Açıklamalar)

- **Feature map (Özellik haritası):**  
  CNN’in bir katmandan ürettiği `[H × W]` boyutundaki haritalardır. Kanal sayısı kadar feature map bulunur.

- **Kanal (Channel):**  
  Aynı görüntünün farklı “öğrenilmiş özellik” temsilleri. RGB gibi sabit değildir; ağ tarafından öğrenilir.

- **Residual / Skip Connection:**  
  \[
  y = x + F(x)
  \]  
  yapısıdır. Gradyan akışını kolaylaştırır ve derin ağların daha stabil eğitilmesini sağlar.

- **Identity (Skip):**  
  Boyutlar (kanal ve uzamsal) uyuyorsa skip hattında doğrudan `x`’in kullanılmasıdır.

- **Projection (Downsample):**  
  Boyutlar uymuyorsa skip hattında `1×1 Conv (+ BN)` kullanılarak kanal ve/veya uzamsal boyut eşlemesi yapılmasıdır.

- **Stride:**  
  Konvolüsyonun adım boyu.  
  `stride = 2` → uzamsal boyutlar (`H × W`) yaklaşık yarıya düşer.

- **Groups (Cardinality):**  
  3×3 konvolüsyonun kanalları kaç parçaya bölerek çalıştığını belirtir.  
  ResNeXt mimarisinin ana ölçekleme eksenidir.

- **Grouped Convolution:**  
  Kanalların `groups` adet gruba ayrılıp her grubun kendi konvolüsyonunu yapmasıdır.  
  Çoğunlukla sonrasında `1×1 Conv` ile gruplar tekrar birleştirilir.

- **base_width:**  
  Her grup için taban genişlik ölçeğidir.  
  `width` hesabında kullanılır ve ResNeXt’in iç kanal düzenini belirler.

- **width (Internal Width):**  
  Grouped `3×3` konvolüsyonun çalıştığı **toplam iç kanal sayısıdır**.

- **expansion:**  
  Son `1×1 Conv`’un kanalı kaç kat genişlettiğini belirler.  
  ResNeXt / ResNet standartlarında genellikle `4` olarak kullanılır.

- **bottleneck_ch (Planes):**  
  Stage’in temel kanal parametresidir.  
  Çıkış kanalı şu şekilde belirlenir:  
  \[
  out\_ch = bottleneck\_ch \times expansion
  \]
